In [1]:
import os
import numpy as np
import mne

from repo import SubjectData

In [2]:
os.environ["TASK"] = 'baseline'
os.environ["MAT"] = 's03'

In [3]:
import logging
logging.basicConfig(filename=f'logs/{os.environ["TASK"]}.log', level=logging.DEBUG)

In [14]:
sd = SubjectData(f'data/{os.environ["MAT"]}')
logging.info(f'File: {os.environ["MAT"]}')

Creating RawArray with float64 data, n_channels=69, n_times=358400
    Range : 0 ... 358399 =      0.000 ...   699.998 secs
Ready.
Creating RawArray with float64 data, n_channels=69, n_times=358400
    Range : 0 ... 358399 =      0.000 ...   699.998 secs
Ready.
Creating RawArray with float64 data, n_channels=69, n_times=71680
    Range : 0 ... 71679 =      0.000 ...   139.998 secs
Ready.
Creating RawArray with float64 data, n_channels=69, n_times=71680
    Range : 0 ... 71679 =      0.000 ...   139.998 secs
Ready.
Creating RawArray with float64 data, n_channels=68, n_times=33920
    Range : 0 ... 33919 =      0.000 ...    66.248 secs
Ready.
Creating RawArray with float64 data, n_channels=68, n_times=5120
    Range : 0 ... 5119 =      0.000 ...     9.998 secs
Ready.
Creating RawArray with float64 data, n_channels=68, n_times=5120
    Range : 0 ... 5119 =      0.000 ...     9.998 secs
Ready.
Creating RawArray with float64 data, n_channels=68, n_times=5120
    Range : 0 ... 5119 =      0.

In [15]:
tmin = -1
tmax = 4
reject_criteria = {'eeg': 350e-6}       # 150 µV The default from the overview tutorial
filter_freqs = (7, 30)
filter_props = dict(picks=['eeg'], fir_design='firwin', skip_by_annotation='edge')

In [16]:
imagery_events = mne.find_events(sd.raw_imagery_left, stim_channel=sd.stim_channel)   # Same stim for imagery l/r
im_left_epochs = mne.Epochs(
    sd.raw_imagery_left, imagery_events, tmin=tmin, tmax=tmax, preload=True
).filter(*filter_freqs, **filter_props).drop_bad(reject_criteria)
im_right_epochs = mne.Epochs(
    sd.raw_imagery_right, imagery_events, tmin=tmin, tmax=tmax,preload=True
).filter(*filter_freqs, **filter_props).drop_bad(reject_criteria)

100 events found
Event IDs: [1]
Not setting metadata
Not setting metadata
100 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 100 events and 2561 original time points ...
0 bad epochs dropped
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 845 samples (1.650 sec)

0 bad epochs dropped
Not setting metadata
Not setting metadata
100 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 100 events and 2561 original time points ...
0 bad

In [17]:
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.pipeline import Pipeline

from mne.decoding import CSP

In [18]:
#im_left_epochs = im_left_epochs#.copy().crop(tmin=1., tmax=2.)   # Not sure why the guy cropped this
im_left_data = im_left_epochs.copy().pick_types(eeg=True)
im_right_data = im_right_epochs.copy().pick_types(eeg=True)
im_left_labels = im_left_epochs.events[:, -1] - 1   # Label: 0
im_right_labels = im_right_epochs.events[:, -1]     # Label: 1

im_data = np.vstack((im_left_data.get_data(), im_right_data.get_data()))
im_labels = np.hstack((im_left_labels, im_right_labels))

print('Left Imagery Data Shape:', im_left_data.get_data().shape)
print('Right Imagery Data Shape:', im_right_data.get_data().shape)
logging.info(f'Left Imagery Data Shape: {im_left_data.get_data().shape}')
logging.info(f'Right Imagery Data Shape: {im_right_data.get_data().shape}')

Left Imagery Data Shape: (100, 64, 2561)
Right Imagery Data Shape: (100, 64, 2561)


In [19]:
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, im_data, im_labels, cv=cv, n_jobs=8)

class_balance = np.mean(im_labels == im_labels[0])
class_balance = max(class_balance, 1. - class_balance)

In [20]:
print("Classification accuracy:", np.mean(scores))
print("Class balance:", class_balance)
logging.info(f"Classification scores: {list(scores)}")
logging.info(f"Classification accuracy: {np.mean(scores)}")
logging.info(f"Class balance: {class_balance}")

Classification accuracy: 0.8125
Class balance: 0.5
